In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from time import clock

In [2]:
basePlayerUrl = "https://www.redkaraoke.com";
baseS3Url = "https://s3.amazonaws.com/";

In [8]:
links_list = []
for i in range(10):
    r = requests.get('https://www.redkaraoke.com/karaoke/celine-dion/my-heart-will-go-on/11827/recordings/p' + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all('a', attrs = {'class':'result result-recording'}):
        links_list.append('https://www.redkaraoke.com' + link.get('href'))
# Remove duplicated links
links_list_unique = set(links_list)
links_list_unique

{'https://www.redkaraoke.com/profile/Allira93/recordings/343699',
 'https://www.redkaraoke.com/profile/AngelJamae/recordings/664612',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/325316',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/342056',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/342084',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/356458',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/368931',
 'https://www.redkaraoke.com/profile/Angelsoft/recordings/386275',
 'https://www.redkaraoke.com/profile/Anti_Suwaryo/recordings/728780',
 'https://www.redkaraoke.com/profile/Arena55/recordings/648309',
 'https://www.redkaraoke.com/profile/BERNITA/recordings/1305787',
 'https://www.redkaraoke.com/profile/Bea18/recordings/624790',
 'https://www.redkaraoke.com/profile/CallieFord/recordings/65921',
 'https://www.redkaraoke.com/profile/CarmenDeMaria/recordings/1927954',
 'https://www.redkaraoke.com/profile/EnioPedrozo/recordings/3

In [9]:
print(len(links_list))
print(len(links_list_unique))

2400
122


In [10]:
# Features in javascript object "userRecording" :
# features = ['file','title','u','uavatar','ugroup','a','rid','adv']
# features1 = ['no_of_views','no_of_likes','no_of_comments','no_of_recordings','no_of_followers','no_of_following','date_recorded','gender_loc']
# features2 = ['recording_id','user_id','user_name','recording_url','file_url_tail','user_group']

counter = 0

#Initialize lists
view_list = []
like_list = []
comment_list = []
recording_list = []
follower_list = []
following_list = []
date_list = []
gender_loc_list = []

recording_id_list = []
user_name_list = []
recording_url_list = []
user_group_list = []
file_url_tail_list = []

# Send request for each link in links_list_unique
for link_sub in links_list_unique:
    r = requests.get(link_sub)
    soup = BeautifulSoup(r.content, 'lxml')
    # Localize data in "userRecording" object

    try:
        counter = counter + 1
        
        # Extract information from html code:
        view = int(soup.find('li', attrs = {'class':'rec-views'}).text)
        like = soup.find('li', attrs = {'class':'rec-likes'}).text
        comment = int(soup.find('li', attrs = {'class':'rec-comments'}).text)
        recording = int(soup.find('li', attrs = {'class':'tam1'}).find('strong').text)
        follower = int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[5].find('strong').text)
        following = (soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[6].find('strong').text)
        date = soup.find_all('div', attrs = {'class':'infoTema_textoBlanco'})[2].text[3:]
        gender_loc = soup.find('div', attrs = {'class':'userdata'}).find('p').text

        recording_url = link_sub

        pattern = re.compile(r"userRecording")
        data = soup.find('script', text=pattern)

#         string_head = 'userRecording.rid='
#         # Take the tail which contains the information needed
#         full_string = re.search(string_head + '.*"',str(data)).group(0)
#         recording_id = full_string[len(string_head)+1:-1]         

#         string_head = 'userRecording.u='
#         # Take the tail which contains the information needed
#         full_string = re.search(string_head + '.*"',str(data)).group(0)
#         user_name = full_string[len(string_head)+1:-1] 
        
#         string_head = 'userRecording.ugroup='
#         # Take the tail which contains the information needed
#         full_string = re.search(string_head + '.*"',str(data)).group(0)
#         user_group = full_string[len(string_head)+1:-1] 

#         string_head = 'userRecording.file='
#         # Take the tail which contains the information needed
#         full_string = re.search(string_head + '.*"',str(data)).group(0)
#         file_url_tail = full_string[len(string_head)+1:-1] 

        pattern = re.compile(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);")
        data = soup.find('script', text=pattern)
        ur_string = re.findall(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);",str(data))
        ur_dict = dict(ur_string)
        for key in ur_dict.keys():
            ur_dict[key] = ur_dict[key][1:-1]

        isDueto = False

        if 'isDueto' in ur_dict.keys(): 
            isDueto = True
        else:
            isDueto = False

        if ur_dict['file'][-1] == '3':
            if ur_dict['lang'] == 'es':
                if isDueto:
                    filepath = basePlayerUrl + "/data/duets/mp3/es/"
                elif 's3' in ur_dict.keys():
                    filepath = baseS3Url + "redkaraoke/data/mp3/"
                else:
                    filepath = basePlayerUrl + "/data/mp3/es/"
            elif ur_dict['lang'] == 'jp':
                if isDueto:
                    filepath = basePlayerUrl + "/data/duets/mp3/jp/"
                elif 's3' in ur_dict.keys():
                    filepath = baseS3Url + "redkaraokejp/data/mp3/"
                else:
                    filepath = baseS3Url + "/data/mp3/jp/"
            else:
                if isDueto:
                    filepath = basePlayerUrl + "/data/duets/mp3/en/"
                elif 's3' in ur_dict.keys():
                    filepath = baseS3Url + "redkaraokecom/data/mp3/"
                else:
                    filepath = basePlayerUrl + "/data/mp3/en/"
        else:
            if ur_dict['lang'] == 'es':
                if isDueto:
                    filepath = basePlayerUrl + "/data/duets/vid/es/"
                elif 'isDueto' in ur_dict.keys():
                    filepath = baseS3Url + "redkaraoke/data/vid/"
                else:
                    filepath = basePlayerUrl + "/data/vid/es/"
            elif ur_dict['lang'] == 'jp':
                if isDueto:
                    filepath = basePlayerUrl + "/data/duets/vid/jp/"
                elif 's3' in ur_dict.keys():
                    filepath = baseS3Url + "redkaraokejp/data/vid/"
                else:
                    filepath = baseS3Url + "/data/vid/jp/"
            else:
                if isDueto:
                    filepath = basePlayerUrl + "/data/duets/vid/en/"
                elif 's3' in ur_dict.keys():
                    filepath = baseS3Url + "redkaraokecom/data/vid/"
                else:
                    filepath = basePlayerUrl + "/data/vid/en/"
        full_link = filepath + ur_dict['file']

        print(full_link)            
            
    except (IndexError, AttributeError) as e:
        continue
        
    view_list.append(view)
    like_list.append(like)
    comment_list.append(comment)
    recording_list.append(recording)
    follower_list.append(follower)
    following_list.append(following)
    date_list.append(date)
    gender_loc_list.append(gender_loc)
    
#     recording_url_list.append(recording_url)
#     recording_id_list.append(recording_id)
#     user_name_list.append(user_name)
#     user_group_list.append(user_group)
#     file_url_tail_list.append(file_url_tail)

    print(counter)

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/fc8f815103c99381d6cd8c1e9fd24d42_1.mp4
1
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/d64fab138917a62058598317c17f896c_1.mp3
2
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/a57e72d5b77085d51fc6260a01fcdbe3_1.mp3
3
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/18f09851159d50a965202df2a1e10622_1.mp4
4
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9f5937efb94d8a17d44ed3f671ab174d_1.mp4
5
https://www.redkaraoke.com/data/vid/es/11827/fc76332ffc78de4be952b790f50ed166_1.mp4
6
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/286d52526147e2f3df99e9c612298602_1.mp3
7
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/f752ecd48329fe9e8e406d2e24a101f9_1.mp4
8
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/f339b92ffc07dd1c520443543bc88044_1.mp4
9
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/9fe9ac9350ded7caa9e66f38fdb4187d_1.mp4
11
https://www.redkaraoke.com/data/vid/es/11827/bd83b51f5bab74bf6c7

https://s3.amazonaws.com/redkaraokecom/data/vid/11827/45ce0bdd615d9358d8e5dbc8816219c8_1.mp4
91
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/10e7d8dc59dcf491519bf163f5d28813_1.mp4
92
https://s3.amazonaws.com/redkaraoke/data/mp3/11827/b6a4117bd3e4db1bd7eaa5b390a9c6aa_1.mp3
93
https://s3.amazonaws.com/redkaraokecom/data/mp3/11827/2f69f038e15f5fb8d1e39a91fbd9db5f_1.mp3
94
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/ad33b5c89cc5d100a86c42919192c06c_1.mp4
95
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/113e5eedb1d975aebb6e817e7d9bba92_1.mp4
96
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/20ed2f22fe6dbfdfa5ff7dc3983b0979_1.mp4
97
https://www.redkaraoke.com/data/mp3/es/11827/63735e1c969fd219c4b0d30b5ce592aa_1.mp3
98
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/c82aa1cd3636240845931df1492d514e_1.mp4
99
https://s3.amazonaws.com/redkaraokecom/data/vid/11827/8f7650f7dfa768667a0c11bda16ea500_1.mp4
100
https://s3.amazonaws.com/redkaraokejp/data/vid/1182

In [ ]:
d = {'recording_id': recording_id_list, 'user_name': user_name_list, 'url_recording': recording_url_list,
     'user_group': user_group_list, 'url_file_tail': file_url_tail_list, 'no_of_views': view_list, 
     'no_of_likes': like_list, 'no_of_comments': comment_list, 'no_of_recordings': recording_list,
     'no_of_followers': follower_list, 'no_of_following': following_list, 'date_recorded': date_list,
     'gender + location': gender_loc_list, 'url_file': file_url}

In [ ]:
df = pd.DataFrame(data=d)

In [ ]:
df

In [ ]:
df.to_csv('./data/out.csv',index=False)

In [4]:
link_sub = 'https://www.redkaraoke.com/profile/Jewel_Emi/recordings/11038'
r = requests.get(link_sub)
soup = BeautifulSoup(r.content, 'lxml')

pattern = re.compile(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);")
data = soup.find('script', text=pattern)
ur_string = re.findall(r"userRecording\.([a-zA-Z0-9]+)=([^;]+);",str(data))
ur_dict = dict(ur_string)

for key in ur_dict.keys():
    ur_dict[key] = ur_dict[key][1:-1]

ur_dict

{'a': 'Celine Dion',
 'adv': 'true',
 'allowfullscreen': 'true',
 'allowscriptaccess': 'always',
 'callback': 'https://www.redkaraoke.com/pc.php?s=11827&si=es&tp=2',
 'file': '11827/8cdcce5e9807c1ad6d6c11a014c7c5a4_1.mp3',
 'height': '480',
 'l': '1',
 'lang': 'jp',
 'loc': 'en',
 'rid': '11038',
 's3': '1',
 'showeq': 'true',
 'sid': '11827',
 'title': 'My heart will go on',
 'u': 'Jewel_Emi',
 'uavatar': 'https://d4rgvppl55fks.cloudfront.net/forum/customprofilepics/profilepic325125_3.gif',
 'ugroup': '9',
 'width': '640',
 'wmode': 'transparent'}

In [6]:
isDueto = False

if 'isDueto' in ur_dict.keys(): 
    isDueto = True
else:
    isDueto = False

if ur_dict['file'][-1] == '3':
    if ur_dict['lang'] == 'es':
        if isDueto:
            filepath = basePlayerUrl + "/data/duets/mp3/es/"
        elif 's3' in ur_dict.keys():
            filepath = baseS3Url + "redkaraoke/data/mp3/"
        else:
            filepath = basePlayerUrl + "/data/mp3/es/"
    elif ur_dict['lang'] == 'jp':
        if isDueto:
            filepath = basePlayerUrl + "/data/duets/mp3/jp/"
        elif 's3' in ur_dict.keys():
            filepath = baseS3Url + "redkaraokejp/data/mp3/"
        else:
            filepath = baseS3Url + "/data/mp3/jp/"
    else:
        if isDueto:
            filepath = basePlayerUrl + "/data/duets/mp3/en/"
        elif 's3' in ur_dict.keys():
            filepath = baseS3Url + "redkaraokecom/data/mp3/"
        else:
            filepath = basePlayerUrl + "/data/mp3/en/"
else:
    if ur_dict['lang'] == 'es':
        if isDueto:
            filepath = basePlayerUrl + "/data/duets/vid/es/"
        elif 'isDueto' in ur_dict.keys():
            filepath = baseS3Url + "redkaraoke/data/vid/"
        else:
            filepath = basePlayerUrl + "/data/vid/es/"
    elif ur_dict['lang'] == 'jp':
        if isDueto:
            filepath = basePlayerUrl + "/data/duets/vid/jp/"
        elif 's3' in ur_dict.keys():
            filepath = baseS3Url + "redkaraokejp/data/vid/"
        else:
            filepath = baseS3Url + "/data/vid/jp/"
    else:
        if isDueto:
            filepath = basePlayerUrl + "/data/duets/vid/en/"
        elif 's3' in ur_dict.keys():
            filepath = baseS3Url + "redkaraokecom/data/vid/"
        else:
            filepath = basePlayerUrl + "/data/vid/en/"
full_link = filepath + ur_dict['file']

full_link

'https://s3.amazonaws.com/redkaraokejp/data/mp3/11827/8cdcce5e9807c1ad6d6c11a014c7c5a4_1.mp3'

In [ ]:
# function ActionTestDueto() {
	
# 	if (duet != null) { isDueto = true; } else { isDueto = false; }	
	
# function ActionTheFile() {
# 	var filepath;
# 	if (/\.mp3/.test(playFile)) {
# 		if (userCountry == "es") {
# 			if (isDueto) {
# 				filepath = basePlayerUrl + "/data/duets/mp3/es/";
# 			} else if (s3 == 1) {
# 				filepath = baseS3Url + "redkaraoke/data/mp3/";
# 			} else {
# 				filepath = basePlayerUrl + "/data/mp3/es/";
# 			}
# 		} else if (userCountry == "jp") {
# 			if (isDueto) {
# 				filepath = basePlayerUrl + "/data/duets/mp3/jp/";
# 			} else if (s3 == 1) {
# 				filepath = baseS3Url + "redkaraokejp/data/mp3/";
# 			} else {
# 				filepath = basePlayerUrl + "/data/mp3/jp/";
# 			}
# 		} else {
# 			if (isDueto) {
# 				filepath = basePlayerUrl + "/data/duets/mp3/en/";
# 			} else if (s3 == 1) {
# 				filepath = baseS3Url + "redkaraokecom/data/mp3/";
# 			} else {
# 				filepath = basePlayerUrl + "/data/mp3/en/";
# 			}
# 		}
# 	} else {
# 		if (userCountry == "es") {
# 			if (isDueto) {
# 				filepath = basePlayerUrl + "/data/duets/vid/es/";
# 			} else if (s3 == 1) {
# 				filepath = baseS3Url + "redkaraoke/data/vid/";
# 			} else {
# 				filepath = basePlayerUrl + "/data/vid/es/";
# 			}
# 		} else if (userCountry == "jp") {
# 			if (isDueto) {
# 				filepath = basePlayerUrl + "/data/duets/vid/jp/";
# 			} else if (s3 == 1) {
# 				filepath = baseS3Url + "redkaraokejp/data/vid/";
# 			} else {
# 				filepath = basePlayerUrl + "/data/vid/jp/";
# 			}
# 		} else {
# 			if (isDueto) {
# 				filepath = basePlayerUrl + "/data/duets/vid/en/";
# 			} else if (s3 == 1) {
# 				filepath = baseS3Url + "redkaraokecom/data/vid/";
# 			} else {
# 				filepath = basePlayerUrl + "/data/vid/en/";
# 			}
# 		}
# 	}	
# 	return filepath+playFile;
# }

In [ ]:
string_head = 'userRecording.rid='
# Take the tail which contains the information needed
full_string = re.search(string_head + '.*"',str(data)).group(0)
recording_id = full_string[len(string_head)+1:-1] 


In [ ]:
link_sub

In [ ]:
file_url